<a href="https://colab.research.google.com/github/velasoka-repo/EVA4/blob/master/Session-8/EVA4_S8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Github Link](https://github.com/velasoka-repo/EVA4/new/master/Session-8)

# Google Drive Mount

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
EVA4_PATH = "/content/gdrive/My Drive/Colab Notebooks/EVA4/"
import sys
if not EVA4_PATH in sys.path:
  sys.path.append(EVA4_PATH)
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/gdrive/My Drive/Colab Notebooks/EVA4/']

# Python Module Import

In [0]:
import velasoka
device = velasoka.get_available_device()

# ResNet18

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [0]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# Run

In [0]:
#model
net = ResNet(BasicBlock, [2,2,2,2]).to(device)

#dataset, data_loader
dataset = velasoka.get_cifar10_dataset()
data_loader = velasoka.get_cifar10_data_loader(dataset=dataset)

#optimizer, loss_function
optimizer = velasoka.get_sgd_optimizer(model=net)
loss_fn = velasoka.get_ce_loss() #CrossEntropyLoss

#model_summary
velasoka.model_summary(model=net, input_size=(3, 32, 32), device=device)
#start cnn
velasoka.start_cnn(model=net, optimizer=optimizer, loss_fn=loss_fn, data_loader=data_loader, device=device, epochs=8)

#save model
drive_file_path = velasoka.get_drive_path(folder="Session-8", filename="resnet18.pt")
velasoka.save_model(model=net, path=drive_file_path)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


  0%|          | 0/12500 [00:00<?, ?it/s]

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 16, 16]          73,728
      BatchNorm2d-14          [-1, 128,

Training Batch=12499, loss=1.81592, Correct Prediction=25236/50000, Train Accuracy=50.47200: 100%|██████████| 12500/12500 [03:34<00:00, 58.17it/s]
Test Batch=2499, Correct Validation=6603/10000, Test Accuracy=66.03000: 100%|██████████| 2500/2500 [00:25<00:00, 98.42it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 2


...

Training Batch=12499, loss=0.24829, Correct Prediction=43866/50000, Train Accuracy=87.73200: 100%|██████████| 12500/12500 [03:29<00:00, 60.66it/s]
Test Batch=2499, Correct Validation=8275/10000, Test Accuracy=82.75000: 100%|██████████| 2500/2500 [00:24<00:00, 100.32it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 6


Training Batch=12499, loss=0.17316, Correct Prediction=45574/50000, Train Accuracy=91.14800: 100%|██████████| 12500/12500 [03:30<00:00, 59.46it/s]
Test Batch=2499, Correct Validation=8170/10000, Test Accuracy=81.70000: 100%|██████████| 2500/2500 [00:25<00:00, 99.06it/s] 
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 7


Training Batch=12499, loss=0.05450, Correct Prediction=46824/50000, Train Accuracy=93.64800: 100%|██████████| 12500/12500 [03:28<00:00, 59.81it/s]
Test Batch=2499, Correct Validation=8265/10000, Test Accuracy=82.65000: 100%|██████████| 2500/2500 [00:25<00:00, 98.67it/s]


# ResNet18 without Linear (Fully Connected Layer)

In [0]:
class ResNet_WOFC(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet_WOFC, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.conv2 = nn.Conv2d(in_channels=512, out_channels=10, kernel_size=1)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = self.conv2(out)
        out = out.view(-1, 10)
        return out

# Run

In [27]:
#model
net = ResNet_WOFC(BasicBlock, [2,2,2,2]).to(device)

#dataset, data_loader
dataset = velasoka.get_cifar10_dataset()
data_loader = velasoka.get_cifar10_data_loader(dataset=dataset)

#optimizer, loss_function
optimizer = velasoka.get_sgd_optimizer(model=net)
loss_fn = velasoka.get_ce_loss() #CrossEntropyLoss

#model_summary
velasoka.model_summary(model=net, input_size=(3, 32, 32), device=device)
#start cnn
velasoka.start_cnn(model=net, optimizer=optimizer, loss_fn=loss_fn, data_loader=data_loader, device=device, epochs=8)

#save model
drive_file_path = velasoka.get_drive_path(folder="Session-8", filename="resnet18-wo-fc.pt")
velasoka.save_model(model=net, path=drive_file_path)

Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/12500 [00:00<?, ?it/s]

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 16, 16]          73,728
      BatchNorm2d-14          [-1, 128,

Training Batch=12499, loss=1.55201, Correct Prediction=25588/50000, Train Accuracy=51.17600: 100%|██████████| 12500/12500 [03:36<00:00, 57.70it/s]
Test Batch=2499, Correct Validation=6693/10000, Test Accuracy=66.93000: 100%|██████████| 2500/2500 [00:25<00:00, 93.84it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 2


...

Training Batch=12499, loss=0.03348, Correct Prediction=43931/50000, Train Accuracy=87.86200: 100%|██████████| 12500/12500 [03:37<00:00, 57.45it/s]
Test Batch=2499, Correct Validation=8176/10000, Test Accuracy=81.76000: 100%|██████████| 2500/2500 [00:26<00:00, 96.12it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 6


Training Batch=12499, loss=0.31785, Correct Prediction=45606/50000, Train Accuracy=91.21200: 100%|██████████| 12500/12500 [03:34<00:00, 58.16it/s]
Test Batch=2499, Correct Validation=8110/10000, Test Accuracy=81.10000: 100%|██████████| 2500/2500 [00:25<00:00, 96.78it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 7


Training Batch=12499, loss=0.02601, Correct Prediction=46918/50000, Train Accuracy=93.83600: 100%|██████████| 12500/12500 [03:35<00:00, 58.11it/s]
Test Batch=2499, Correct Validation=8257/10000, Test Accuracy=82.57000: 100%|██████████| 2500/2500 [00:26<00:00, 95.87it/s]
